This "app" is just a general function which contains every needed function about cleaning data in MOLONARI's project. The inner functions have been adapted to build the GUI in the "test" file

In [ ]:
def app():
    # Select path for each file
    path_cap_riviere = easygui.fileopenbox()  #P
    path_cap_ZH = easygui.fileopenbox()  #T
    path_etalonnage_capteur_riv = easygui.fileopenbox() #P508

    # Hardcoded path while testing
#     path_cap_riviere = "../MOLONARI_1D_RESOURCES/sampling_points/Point034/point034_P_measures.csv"
#     path_cap_ZH = "../MOLONARI_1D_RESOURCES/sampling_points/Point034/point034_T_measures.csv"
#     path_etalonnage_capteur_riv = "../MOLONARI_1D_RESOURCES/configuration/pressure_sensors/P508.csv"

    #Opening files as DataFrames
    capteur_riviere = pd.read_csv(path_cap_riviere,header=1)
    capteur_ZH = pd.read_csv(path_cap_ZH,header=1)
    etalonnage_capteur_riv = pd.read_csv(path_etalonnage_capteur_riv)
    
    # Set seed for reproducibility
    np.random.seed(0)
    
    # Drop last columns. Not valuable data.
    capteur_riviere.drop(labels = list(capteur_riviere.columns)[4:],axis=1,inplace=True)
    capteur_ZH.drop(labels = list(capteur_ZH.columns)[6:],axis=1,inplace=True)

    # Rename columns as requested
    capteur_riviere.rename(columns={'#': '#', list(capteur_riviere.columns)[1]: 'dates',list(capteur_riviere.columns)[2]: 'tension_V',
                                    list(capteur_riviere.columns)[3]:'temperature_stream_C'}, inplace =  True)
    capteur_ZH.rename(columns={'#': '#', list(capteur_ZH.columns)[1]: 'dates', list(capteur_ZH.columns)[2]: 'temperature_depth_1_C',
                               list(capteur_ZH.columns)[3]:'temperature_depth_2_C', list(capteur_ZH.columns)[4]:'temperature_depth_3_C',
                               list(capteur_ZH.columns)[5]:'temperature_depth_4_C'}, inplace =  True)
    
    #Getting the charge difference in metters following "charge_m = (tens_V-temp_C·dU/dT-Intercept)/(dU/dH)"
    intercept = float(etalonnage_capteur_riv.iloc[2][list(etalonnage_capteur_riv.columns)[-1]])
    dUdH = float(etalonnage_capteur_riv.iloc[3][list(etalonnage_capteur_riv.columns)[-1]])
    dUdT = float(etalonnage_capteur_riv.iloc[4][list(etalonnage_capteur_riv.columns)[-1]])
    capteur_riviere["charge_m"] = (capteur_riviere["tension_V"]-capteur_riviere["temperature_stream_C"]*dUdT-intercept)/dUdH
    
    #converting dates in datetime64
    capteur_riviere['dates'] = pd.to_datetime(capteur_riviere['dates'], infer_datetime_format=True)
    capteur_ZH['dates'] = pd.to_datetime(capteur_ZH['dates'], infer_datetime_format=True)
    
    def verify_days(sensor):
        # Shows an histogram to verify that the quantity of data for each day makes sense (days must be between 1 and 31)
        # sensor : sensor DataFrame with a column named "dates" and type "datetime64"
        # Ex. : verify_days(capteur_riviere)
        days = sensor["dates"].dt.day
        plt.hist(days,range=[1,32],bins=31)
    
    def box_plot(sensor):
        # Shows a box plot for each of the sensor variables
        # sensor : sensor DataFrame with data of the variables from the third to the last column
        # Ex. : box_plot(capteur_ZH)
        fig, axs = plt.subplots(1, len(list(sensor.columns)[2:]),figsize=(20,5))
        for i in range(len(list(sensor.columns)[2:])):
            sensor.boxplot(column = list(sensor.columns)[2+i], ax=axs[i])
        plt.show()
     
    def histogram(sensor):
        # Shows an histogram for each of the sensor variables
        # sensor : sensor DataFrame with data of the variables from the third to the last column
        # Ex. : histogram(capteur_ZH)
        sensor[list(sensor.columns)[2:]].hist(bins=100,figsize=(20,4),layout=(1,len(list(sensor.columns)[2:])))
        plt.subplots_adjust(wspace=0.2,hspace=0.2)
        plt.show()
    
    def scatterplot(var1, var2):
        # Shows a scatterplot relating var1 and var2
        # var1, var2 : each one is a Series with the data of the variable.
        # Ex. : scatterplot(capteur_riviere["temperature_stream_C"], capteur_ZH['temperature_depth_3_C'])
        df = pd.concat([var1, var2], axis=1)
        df.plot.scatter(x = var1.name, y = var2.name)
        plt.subplots_adjust(wspace=0.2,hspace=0.2)
        plt.show()
    
    def big_scatterplot(sensor1, sensor2):
        # Shows a big scatterplot where all the variables of both sensors are related
        # sensor1, sensor2 : sensors DataFrame with data of the variables from the third to the last column
        # Ex. : big_scatterplot(capteur_riviere,capteur_ZH)
        var_number = 7
        fig_size = 24
        s1 = sensor1[list(sensor1.columns)[2:]].copy()
        s2 = sensor2[list(sensor2.columns)[2:]].copy()
        df = pd.concat([s1,s2], axis=1)
        fig, axs = plt.subplots(var_number, var_number,figsize=(fig_size,fig_size))
        for i in range(var_number):
            for j in range(var_number):
                df.plot.scatter(x = list(df.columns)[i], y = list(df.columns)[j],ax=axs[i,j])
        plt.subplots_adjust(wspace=0.32,hspace=0.32)
        plt.show()

    def get_df(capteur_riviere, capteur_ZH):
        s1 = capteur_riviere[list(capteur_riviere.columns)[3:]].copy()
        s2 = capteur_ZH[list(capteur_ZH.columns)[1:]].copy()
        df = pd.concat([s2,s1], axis=1)
        return df
        
    def remove_outlier_col_iqr(df_in, col_name):
        q1 = df_in[col_name].quantile(0.25)
        q3 = df_in[col_name].quantile(0.75)
        iqr = q3-q1 #Interquartile range
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
        return df_out
    
    def remove_outlier_df_iqr(cap_riv, cap_ZH):
        df = get_df(cap_riv, cap_ZH)
        df_or = df.copy()
        for i in range(1,len(list(df.columns))): 
            df = remove_outlier_col_iqr(df,list(df.columns)[i])
            
        return df, df_or
    
    def remove_outlier_z(cap_riv, cap_ZH):
        df = get_df(cap_riv, cap_ZH)
        df_or = df.copy()
        df.dropna(inplace = True)
        df = df[(np.abs(stats.zscore(df.iloc[:,1:])) < 3).all(axis=1)]
#         df = df[(np.abs(stats.zscore(df.iloc[:,[1,2,3,4,6]])) < 3).all(axis=1)] # Just to compare with Felix's code
        return df, df_or
                
    def plot_outliers(df, df_or):
        df1 = df_or.copy()
        df1.dropna(inplace = True)
        result = pd.concat([df_or, df], axis=1)
        df1['outliers'] = result[list(df1.columns)[1]].iloc[:,1]
        df1['outliers'][np.isnan(df1['outliers'])] = True
        df1['outliers'][df1['outliers'] != True] = False
        fig, axs = plt.subplots(4, 2,figsize=(10,20))
        first = df1["dates"].iloc[0]
        last = df1["dates"].iloc[-1]
        for i in range(1,len(list(df1.columns))-1):
            a = math.floor((i-1)/2)
            b = (i-1)%2
            df1[df1['outliers'] == False].plot(x='dates',y=list(df1.columns)[i],ax = axs[a,b])
            df1[df1['outliers'] == True].plot.scatter(x='dates',y=list(df1.columns)[i],c = 'r',s = 3,ax = axs[a,b])
            axs[a,b].tick_params(labelrotation=30)
            axs[a,b].set_xlim([datetime.date(first.year, first.month, first.day), datetime.date(last.year, last.month, last.day+1)])

        plt.subplots_adjust(wspace=0.3,hspace=0.5)
        plt.show()
        
    
        
    
        
#     df_iqr, df_or_iqr = remove_outlier_df_iqr(capteur_riviere, capteur_ZH)
#     plot_outliers(df_iqr, df_or_iqr)
#     df_z, df_or_z = remove_outlier_z(capteur_riviere, capteur_ZH)
#     plot_outliers(df_z, df_or_z)
    
app()